In [2]:
import json
import numpy as np
import sqlite3
import cv2 as cv

In [3]:
f = open('passpunkte.json')
data = json.load(f)

In [4]:
db = sqlite3.connect(':memory:')
cur = db.cursor()
db.execute("""CREATE TABLE IF NOT EXISTS passpunktpos (
            ppid INTEGER PRIMARY KEY AUTOINCREMENT,
            pid INTEGER,
            bid INTEGER,
            x NUMBER,
            y NUMBER,
            UNIQUE (pid, bid))""")

In [5]:
data['passpunkte']
for pp in data["passpunkte"]:
     cur.execute("INSERT OR REPLACE INTO passpunktpos (pid, bid, x, y) VALUES (?,?,?,?)", (pp['passpunkt'],pp['image'],pp['x'],pp['y']))

In [6]:
cur.execute("SELECT a.bid, b.bid, count(*) FROM passpunktpos a, passpunktpos b WhERE a.pid = b.pid and a.bid < b.bid GROUP BY a.bid, b.bid ORDER BY count(*) DESC LIMIT 1")
first_pair = cur.fetchone()
first_pair

(0, 1, 8)

In [7]:
cur.execute("SELECT a.pid, a.x, a.y, b.x, b.y FROM passpunktpos a, passpunktpos b WhERE a.pid = b.pid and a.bid = ? AND b.bid = ?", (first_pair[0], first_pair[1]))
coords = cur.fetchall()
coords[0]


(0,
 830.7972047639337,
 2635.0598909840774,
 872.703851701128,
 2417.2588006475785)

In [8]:
pts1 = np.empty((len(coords),2))
pts2 = np.empty((len(coords),2))

for i, pp in enumerate(coords):
    pid, x1, y1, x2, y2 = pp
    pts1[i, 0] = x1
    pts1[i, 1] = y1
    pts2[i, 0] = x2
    pts2[i, 1] = y2

pts1

array([[ 830.79720476, 2635.05989098],
       [3284.37680885, 2735.66436457],
       [1539.64102884, 1280.31160593],
       [3130.14047711,  146.48328435],
       [2810.05859375, 1377.44140625],
       [ 248.87815065, 2098.11323308],
       [3167.96875   , 1501.953125  ],
       [2748.34904925, 1763.32293725]])

In [9]:
K = np.array([[3000,0,2000],
            [0,3000,1500],
            [0,0,1]])
K

array([[3000,    0, 2000],
       [   0, 3000, 1500],
       [   0,    0,    1]])

In [10]:
F, mask = cv.findFundamentalMat(pts1, pts2, cv.FM_RANSAC)
F

array([[ 2.76474530e-09, -1.03827651e-06,  3.86519042e-03],
       [ 1.23498682e-06, -1.58117677e-08, -2.26770667e-03],
       [-3.63630580e-03,  1.56948786e-03,  1.00000000e+00]])

In [15]:
E = K.T@F@K
E/E[0,1]

array([[-0.00266282,  1.        , -0.74267472],
       [-1.1894585 ,  0.01522886, -0.05732234],
       [ 0.57091281,  0.17040509, -0.10445242]])

In [12]:
U, S, V = np.linalg.svd(E)

# Ensure rotation matrix are right-handed with positive determinant
if np.linalg.det(np.dot(U, V)) < 0:
    V = -V

# create 4 possible camera matrices (Hartley p 258)
W = np.array([[0, -1, 0], [1, 0, 0], [0, 0, 1]])
P2s = [np.vstack((np.dot(U, np.dot(W, V)).T, U[:, 2])).T,
      np.vstack((np.dot(U, np.dot(W, V)).T, -U[:, 2])).T,
      np.vstack((np.dot(U, np.dot(W.T, V)).T, U[:, 2])).T,
      np.vstack((np.dot(U, np.dot(W.T, V)).T, -U[:, 2])).T]

P2s

[array([[ 0.99168087, -0.06153181, -0.1130614 , -0.15817903],
        [ 0.03874822,  0.98030558, -0.19364801,  0.42755438],
        [ 0.12275023,  0.1876561 ,  0.97453454,  0.89004306]]),
 array([[ 0.99168087, -0.06153181, -0.1130614 ,  0.15817903],
        [ 0.03874822,  0.98030558, -0.19364801, -0.42755438],
        [ 0.12275023,  0.1876561 ,  0.97453454, -0.89004306]]),
 array([[-0.98186012, -0.12698248, -0.14080538, -0.15817903],
        [-0.0652935 , -0.47075568,  0.87984422,  0.42755438],
        [-0.17800973,  0.87307762,  0.4539251 ,  0.89004306]]),
 array([[-0.98186012, -0.12698248, -0.14080538,  0.15817903],
        [-0.0652935 , -0.47075568,  0.87984422, -0.42755438],
        [-0.17800973,  0.87307762,  0.4539251 , -0.89004306]])]